In [6]:
from gensim.summarization import keywords as k
from elasticsearch import Elasticsearch

In [7]:
es = Elasticsearch([{'host':'nyuvis-web.poly.edu', 'port':80, 'url_prefix':
                    'es'},])

In [8]:
wo_abs = es.count(index='nips_papers', body={
    'query': {
        'match': {
            'abstract': 'Abstract Missing'
        }
    }
})

In [9]:
tot = es.count(index='nips_papers')

In [12]:
#keywords for document abstracts

doc_list = []
tot_count_abstracts = tot['count']-wo_abs['count']

#curr_year = 2007
temp = []

res = es.search(index='nips_papers', body = {
    "_source": ['id', 'year', 'abstract'],
    "size": tot_count_abstracts,
    "query": {
        "bool": {
            "must_not": {
                "match": {
                    "abstract": "Abstract Missing"
                }
            }
        }
    }
})

#sampling code
'''while curr_year != 2018:
    #10 samples of documents each year
    res = es.search(index = 'nips_papers', body = {
        "_source": ['id', 'year', 'abstract'],
        "size": 20,
        "query": {
            "bool": {
                "must_not": {
                    "match": {
                        "abstract": "Abstract Missing"
                    }
                },
                "must": {
                    "match": {
                        "year": str(curr_year)
                    }
                }
            }
        }
    })
    
    #append the documents of curr_year to the document list
    for entry in res['hits']['hits']:
        doc_list.append(entry['_source'])
    
    if len(res['hits']['hits']) != 0:
        temp.append({curr_year:len(res['hits']['hits'])})
    
    curr_year += 1'''

for entry in res['hits']['hits']:
    doc_list.append(entry['_source'])

len(doc_list)

3808

In [13]:
abs_keyw_list = []

for doc in doc_list:
    keywords = k(doc['abstract'], split = True)
    for word in keywords:
        abs_keyw_list.append({
            'keyword': word,
            'doc_id': doc['id'],
            'year': doc['year']
        })

len(abs_keyw_list)

KeyboardInterrupt: 

In [73]:
import csv

with open('TR_abstract_nips_v1.csv', 'w', newline='') as f:
    header_present = False
    for doc in abs_keyw_list:
        if not header_present:
            w = csv.DictWriter(f, doc.keys())
            w.writeheader()
            header_present = True
        w.writerow(doc)